In [5]:
import tensorflow as tf
from tensorflow import keras
from plantcv import plantcv as pcv
import pandas as pd

In [6]:
print(tf.__version__)
print(pd.__version__)

2.11.0
1.5.3


In [21]:
# How many parameters are we inputing into our model
num_parameters = 13 

In [8]:
# See if there are any GPUs that can be used to run our code
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [26]:
csv_filepath = "/Users/shilo/Documents/GitHub/BiomassPredictonAI/harvest_bydate.csv"

data = pd.read_csv(csv_filepath)
input_data = data.iloc[:,9:]
input_data

,length_mm,width_mm,height_mm,plant_area,plant_convex_hull_area,plant_solidity,plant_perimeter,plant_width,plant_height,plant_longest_path
0,43.07,51.99,5.8,1211,1456.0,0.831731,189.923880,49,42,346
1,52.71,57.06,5.5,1412,1556.5,0.907164,181.338094,56,41,387
2,72.92,64.96,8.9,1323,1766.5,0.748939,225.480229,49,54,332
3,58.18,75.92,7.5,1621,1787.0,0.907107,195.580734,44,59,395
4,71.82,79.79,7.1,1919,2372.0,0.809022,263.421354,62,58,453
...,...,...,...,...,...,...,...,...,...,...
79,96.13,105.59,9.5,3590,4191.5,0.856495,313.279218,88,68,710
80,99.26,105.96,9.4,2832,3384.0,0.836879,272.007140,57,75,595
81,83.12,96.14,8.2,2714,3456.5,0.785187,320.592927,59,84,579
82,70.66,66.16,6.8,1801,2538.5,0.709474,279.178713,74,53,608


In [28]:
output_data = data.iloc[:, 6]
output_data

0     1.30
1     2.10
2     3.36
3     3.07
4     3.17
      ... 
79    7.21
80    6.50
81    6.22
82    3.27
83    7.94
Name: LFW_g, Length: 84, dtype: float64

In [38]:
# created callback to prevent overfitting
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.95:
            print("\nReached 95 percent so cancelling training")
            self.model.stop_training = True


callbacks = myCallback()

In [42]:
# This function will be used to create our model
def create_model():
    model = keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=[10]),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.BatchNormalization(),

    ])

    model.compile(
        loss='Loss', #TODO: change loss function for regression
        optimizer='adam',
        metrics=['accuracy']
    )

    return model

In [43]:
model = create_model()

In [44]:
model.fit(
    input_data,
    output_data,
    epochs=50
)

Epoch 1/50


NotImplementedError: in user code:

    File "/Users/shilo/Documents/GitHub/BiomassPredictonAI/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/shilo/Documents/GitHub/BiomassPredictonAI/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/shilo/Documents/GitHub/BiomassPredictonAI/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/shilo/Documents/GitHub/BiomassPredictonAI/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/shilo/Documents/GitHub/BiomassPredictonAI/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1082, in compute_loss
      

In [ ]:
# This will allow us to save model weights after training
model.save_weights('./checkpoints/model_weights')